In [18]:
import datetime
import os
import time

import ibis

In [5]:
!curl -LsS -o /tmp/geography.db \
    'https://storage.googleapis.com/ibis-tutorial-data/geography.db'

In [6]:
conn = ibis.sqlite.connect('/tmp/geography.db')
conn.list_tables()

['countries', 'gdp', 'independence']

In [7]:
countries = conn.table('countries')

In [8]:
ibis.options.interactive = True

countries['name', 'continent', 'population'].limit(3)

,name,continent,population
0,Andorra,EU,84000
1,United Arab Emirates,AS,4975593
2,Afghanistan,AS,29121286


In [9]:
ibis.options.interactive = False

countries['name', 'continent', 'population'].limit(3)

r0 := AlchemyTable: countries
  iso_alpha2  string
  iso_alpha3  string
  iso_numeric int32
  fips        string
  name        string
  capital     string
  area_km2    float64
  population  int32
  continent   string

r1 := Selection[r0]
  selections:
    name:       r0.name
    continent:  r0.continent
    population: r0.population

Limit[r1, n=3]

In [10]:
countries_expr = countries['name', 'continent', 'population'].limit(3)
type(countries_expr)

ibis.expr.types.relations.Table

In [11]:
countries_expr.execute()

,name,continent,population
0,Andorra,EU,84000
1,United Arab Emirates,AS,4975593
2,Afghanistan,AS,29121286


In [12]:
population_in_millions = (
    countries['population'] / 1_000_000
).name('population_in_millions')
population_in_millions

r0 := AlchemyTable: countries
  iso_alpha2  string
  iso_alpha3  string
  iso_numeric int32
  fips        string
  name        string
  capital     string
  area_km2    float64
  population  int32
  continent   string

population_in_millions: r0.population / 1000000

In [13]:
type(population_in_millions)

ibis.expr.types.numeric.FloatingColumn

In [14]:
countries['name', 'continent', population_in_millions].limit(3)

r0 := AlchemyTable: countries
  iso_alpha2  string
  iso_alpha3  string
  iso_numeric int32
  fips        string
  name        string
  capital     string
  area_km2    float64
  population  int32
  continent   string

r1 := Selection[r0]
  selections:
    name:                   r0.name
    continent:              r0.continent
    population_in_millions: r0.population / 1000000

Limit[r1, n=3]

In [15]:
ibis.options.verbose = True

(countries['name', 'continent', population_in_millions]
 .limit(3)
 .execute())

SELECT t0.name, t0.continent, t0.population / CAST(? AS REAL) AS population_in_millions 
FROM main.countries AS t0
 LIMIT ? OFFSET ?


,name,continent,population_in_millions
0,Andorra,EU,0.084000
1,United Arab Emirates,AS,4.975593
2,Afghanistan,AS,29.121286


In [21]:
def log_query(query: str) -> None:
    path = '/tmp/test.log'
    single_line_query = query.replace('\n', r'\n')
    with open(path, 'a') as f:
        f.write(f'{single_line_query}\n')

In [22]:
ibis.options.verbose_log = log_query

In [23]:
countries.execute()
time.sleep(1.)
(countries['name', 'continent', population_in_millions]
 .limit(3)
 .execute())

,name,continent,population_in_millions
0,Andorra,EU,0.084000
1,United Arab Emirates,AS,4.975593
2,Afghanistan,AS,29.121286


In [24]:
!cat /tmp/test.log

SELECT t0.iso_alpha2, t0.iso_alpha3, t0.iso_numeric, t0.fips, t0.name, t0.capital, t0.area_km2, t0.population, t0.continent \nFROM main.countries AS t0\n LIMIT ? OFFSET ?
SELECT t0.name, t0.continent, t0.population / CAST(? AS REAL) AS population_in_millions \nFROM main.countries AS t0\n LIMIT ? OFFSET ?
